# Transformação do dataset

Neste _notebook_ é realizada a trasnformação do _dataset_ EEG de `.txt` para um arquivo `.npy` (_numpy array_).

In [1]:
import numpy as np

In [ ]:
file1 = '../dataset/OpenBCI-RAW-2023-10-04_17-12-22.txt'
file2 = '../dataset/OpenBCI-RAW-2023-10-25_17-43-36.txt'
file3 = '../dataset/OpenBCI-RAW-2023-11-14_17-25-16.txt'

data1 = np.loadtxt(file1, dtype=float, delimiter=',', skiprows=5, usecols=range(1, 9))
data2 = np.loadtxt(file2, dtype=float, delimiter=',', skiprows=5, usecols=range(1, 9))
data3 = np.loadtxt(file3, dtype=float, delimiter=',', skiprows=5, usecols=range(1, 9))

# data1: 500 pois são 2 segundos iniciais jogados fora (2x250)
# data2: o início foi cortado, bem como o fim (por isso os -1750)
# data3: corte dos 2 segundos iniciais também

data_final = np.concatenate((data1[500:,:], data2[500:-1750,:], data3[500:,:]), axis=0)